In [13]:
import warnings
warnings.filterwarnings('ignore')

In [14]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter


In [15]:
# DB libraries
import psycopg2
from postgres_data import host
from postgres_data import password
from postgres_data import user
from sqlalchemy import create_engine

In [16]:
# ML libraries
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [17]:
# Create an engine instance
alchemyEngine = create_engine(f'postgresql://{user}:{password}@{host}:5432/dfph644rcv92i9')

In [18]:
# Connect to PostgreSQL server
dbConnection = alchemyEngine.connect()

# Read data from PostgreSQL database table and load into a DataFrame instance
df = pd.read_sql("select * from meta", dbConnection)
dbConnection.close()

In [19]:
ml_table = df[['movie_id','release_date','movie_popularity', 
            'vote_average', 'vote_count','budget','revenue', 
            'ratings', 'genres','studios','actor_popularity_mean',
            'director_popularity_mean']]
ml_table = ml_table.drop('movie_id',1)
ml_table

,release_date,movie_popularity,vote_average,vote_count,budget,revenue,ratings,genres,studios,actor_popularity_mean,director_popularity_mean
0,2001-11-16,268.472,7.9,21429,125000000,976475550,PG,"['Adventure', 'Fantasy']","['Warner Bros. Pictures', ' Heyday Films', ' O...",2.993965,2.566
1,2002-05-01,480.954,7.2,14421,139000000,821708551,PG-13,"['Fantasy', 'Action']","['Other', ' Columbia Pictures', ' Sony Picture...",2.387022,2.914
2,2002-11-13,246.027,7.7,17294,100000000,876688482,PG,"['Adventure', 'Fantasy']","['Warner Bros. Pictures', ' Heyday Films', ' O...",2.696712,2.566
3,2004-05-31,225.882,8.0,17001,130000000,789804554,PG,"['Adventure', 'Fantasy']","['Warner Bros. Pictures', ' Other', ' Heyday F...",3.523069,3.333
4,2005-11-16,244.428,7.8,16341,150000000,895921036,PG-13,"['Adventure', 'Fantasy', 'Family']","['Warner Bros. Pictures', ' Heyday Films', ' O...",3.234944,2.695
...,...,...,...,...,...,...,...,...,...,...,...
69,2021-10-14,616.978,6.9,1251,20000000,127000000,R,"['Horror', 'Thriller']","['Universal Pictures', ' Other', ' Other', ' O...",3.464000,2.385
70,2021-10-21,685.843,7.5,458,43000000,19000000,R,"['Horror', 'Mystery', 'Thriller']","['Other', ' Other', ' Other', ' Other', ' Othe...",2.869283,7.468
71,2021-11-03,1746.171,7.1,1170,200000000,336000000,PG-13,"['Action', 'Adventure', 'Science Fiction']",['Marvel Studios'],5.096703,2.397
72,2021-11-11,544.722,7.5,113,75000000,63000000,PG-13,"['Comedy', 'Fantasy']","['Columbia Pictures', ' Other', ' Other', ' Ot...",4.924763,4.999
